In [1]:
import os,sys
modp = os.path.dirname(os.path.abspath(""))
while not "molNet" in os.listdir(modp):
    modp = os.path.dirname(modp)
    if os.path.dirname(modp) == modp:
        raise ValueError("connot determine local molNet")
if modp not in sys.path:
    sys.path.insert(0, modp)
    sys.path.append(modp)

import molNet
import molNet.dataloader.molecular as mol_dl_mod
from molNet.dataloader.molecular.dataloader import MolDataLoader
import inspect
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import molNet.featurizer
from scipy.optimize import curve_fit

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [2]:
BASEDIR=os.path.abspath(os.path.join(molNet.get_user_folder(), "autodata", "feats_raw_filebased"))
OUT_DIR = os.path.abspath(os.path.join(molNet.get_user_folder(), "ecdf_data"))

os.makedirs(OUT_DIR,exist_ok=True)

In [3]:
possible_dataloader=set()
for modname, mod in inspect.getmembers(mol_dl_mod, inspect.ismodule):
    for classname,c in inspect.getmembers(mod, inspect.isclass):
        if issubclass(c,MolDataLoader) and not c==MolDataLoader:
            possible_dataloader.add(c)

found_dataloader_dirs=set()
for pdl in possible_dataloader:
    dataset_name=f"{pdl.__module__}.{pdl.__name__}"
    ds_dir=os.path.join(BASEDIR,dataset_name)
    if os.path.isdir(ds_dir):
        found_dataloader_dirs.add(ds_dir)
found_dataloader_dirs

{'/home/jupyteruser/jupyter_data/molNet_data/autodata/feats_raw_filebased/molNet.dataloader.molecular.ChEMBLdb.ChemBLdb29',
 '/home/jupyteruser/jupyter_data/molNet_data/autodata/feats_raw_filebased/molNet.dataloader.molecular.ESOL.ESOL'}

In [4]:
def argmin1(y1,y2):
    return np.abs(np.subtract.outer(y2,y1)).argmin(1)

def argmin2(y1,y2):
    return np.array([ np.abs(np.subtract(y2[i],y1)).argmin() for i in range(y2.shape[0])])


def generate_ecdf(x, resolution_y=None, smooth=False, unique_only=False):
    if x.ndim > 1:
        x = np.squeeze(x)
        if x.ndim > 1:
            return [
                generate_ecdf(x[..., i], res_1_99=res_1_99, smooth=smooth, unique_only=unique_only)
                for i in range(x.shape[-1])
            ]
    x = np.sort(x)
    x = x[np.isfinite(x)]
    n = len(x)
    y = np.arange(1, n + 1) / n
    if smooth:
        unique_only = True
        x, uindices = np.unique(x, return_index=True)
        y = np.array([a.mean() for a in np.split(y, uindices[1:])])
        y[0] = 0
        y[-1] = 1
    if resolution_y:
        ylin=np.linspace(0,1,resolution_y)
        indices = np.unique(argmin2(y,ylin))
        
        y=y[indices]
        x=x[indices]
        
    if unique_only:
        _x,uindices1 = np.unique(x, return_index=True)
        _x,uindices2 = np.unique(x[::-1], return_index=True)
        uindices2=(len(x)-1)-uindices2
        uindices = np.sort(np.concatenate([uindices1,uindices2]))
        x= x[uindices]
        y = y[uindices]
        
        x,y=np.unique(np.concatenate([x.reshape(1,-1),y.reshape(1,-1)]),axis=1)
        
    return x, y

def try_generate_ecdf(featpath,res=5000,redo=False):
    outpath=os.path.join(OUT_DIR,os.path.relpath(featpath,BASEDIR))
    
    if not os.path.isdir(outpath):
        return False
    histo_data_path=os.path.join(outpath,"histo_data.pckl")
    if not os.path.isfile(histo_data_path):
        return False
    escf_plots=os.path.join(outpath,"escf_plots")
    os.makedirs(escf_plots,exist_ok=True)
    
    ecdf_data_path=os.path.join(outpath,"ecdf_data.pckl")
    while True:
        if redo:
            break
        if not os.path.exists(ecdf_data_path):
            break
            
        with open(ecdf_data_path,"rb") as f:
            ecdf_data = pickle.load(f)
            
        if not isinstance(ecdf_data,dict):
            break
            
        if not "resolution" in ecdf_data:
            break
            
        if ecdf_data["resolution"]!=res:
            break
        
        return True
    
    print("generate ecdf",outpath)
    with open(histo_data_path,"rb") as f:
        histo_data = pickle.load(f)
        
    featurname=os.path.basename(outpath)
    dataset_name=os.path.basename(os.path.dirname(outpath))
    
    titel_wo_i = f"{dataset_name}\n{featurname} {{}}"
    
    
    pp=True
    ecdf_data=[]
     
    for i,d in enumerate(histo_data):
        if len(d["data"])>0:
            if pp:
                print(outpath)
                pp=False
                
            unpacked_data = np.repeat(d["data"], d["counts"], axis=0)
            x1, y1 = generate_ecdf(unpacked_data)
            plt.plot(x1,y1,label="ecdf")

            x3, y3 = generate_ecdf(unpacked_data,smooth=True,resolution_y=res)
            xy=np.concatenate([[x3], [y3]],axis=0)
            ecdf_data.append(xy)
            
            plt.plot(xy[0],xy[1],label="smoothed ecdf")
            plt.title(titel_wo_i.format(i), fontdict = {'fontsize' : 8})
            plt.tight_layout()
            plt.savefig(os.path.join(escf_plots,f"{i}.png"),bbox_inches='tight')
            plt.close()
    with open(ecdf_data_path,"w+b") as f:
        pickle.dump({"data":ecdf_data,"resolution":res},f)
    return True

In [5]:

def featpath_to_histo_data(featpath,histo_data_path):
    
    print("gen histo",featpath)
    counter=[]
    for f in tqdm(os.listdir(featpath),total=len(os.listdir(featpath))):
     
        if not f.startswith("feats_"):
            continue
        try:
            int(f[-5:-4])
        except ValueError:
            continue
        data_fp=os.path.join(featpath,f)

        ignored_fp=os.path.join(featpath,f[:-4]+"_ignored_indices.npy")
        data_array=np.load(data_fp)
        ignored_array=np.load(ignored_fp)
        selector = np.ones(data_array.shape[0],dtype=bool)
        selector[ignored_array]=False
        data_array=data_array[selector]
        #print(data_array)
        for i in range(data_array.shape[1]):
            if len(counter)<=i:
                counter.append({})
            count_dict=counter[i]
            datas,counts = np.unique(data_array[:,i],return_counts=True)
            datas,counts = datas.tolist(),counts.tolist()
            for j in range(len(datas)):
                if datas[j] not in count_dict:
                    count_dict[datas[j]] = 0
                
                count_dict[datas[j]] += counts[j]
    
    np_counter = []
    for c in counter:
        np_counter.append({
            "data":np.array(list(c.keys())),
            "counts":np.array(list(c.values())),
        })
    histo_data = {"histo_data":np_counter,"n_dists":len(np_counter)}
            
    return histo_data
    
        

def check_histo_data_path(histo_data_path,dataloader):
    if not os.path.exists(histo_data_path):
        return False,None
            
    with open(histo_data_path,"rb") as f:
        histo_data = pickle.load(f)

    if not isinstance(histo_data,dict):
        return False,histo_data

    if not "histo_data" in histo_data:
        return False,histo_data
    
    if not "n_dists" in histo_data:
        return False,histo_data
    
    if histo_data["n_dists"] < 1:
        return False,histo_data
    
    for n in range(histo_data["n_dists"]):
        d=histo_data["histo_data"][n]
        if d["counts"].dtype != int:
            return False,histo_data
    
    if "dataloader" not in histo_data:
        histo_data["dataloader"]=[]
    
    for d in dataloader:
        if d not in histo_data["dataloader"]:
            histo_data["dataloader"].append(d)
            
    if len(dataloader)!=len(histo_data["dataloader"]):
        return False,histo_data
    
    return True,histo_data

def check_histodata_images(histo_data,histo_data_path,recreate=False,bin_sizes=[30,50],basetitle="histo"):
    histo_path=os.path.join(histo_data_path,"histos")
    os.makedirs(histo_path,exist_ok=True)
                
    pp=False
    for i in tqdm(range(histo_data["n_dists"]),total=histo_data["n_dists"]):
        path1=os.path.join(histo_path,f"{i}.png")
        
        counts = histo_data["histo_data"][i]["counts"]
        lc=len(counts)
        
        bin_sizes = [b for b in bin_sizes if b<lc]
        
        bin_paths=[os.path.join(histo_path,f"{i}_bin{b}.png") for b in bin_sizes]
        
        
        if os.path.exists(path1) and all([os.path.exists(bp) for bp in bin_paths]) and not recreate:
            continue
        
        if not pp:
            print("gen hist images",histo_data_path)
            pp=True
        bins=histo_data["histo_data"][i]["data"]
        nat_bins=bins[(~np.isneginf(bins))&(~np.isinf(bins))]
        if len(nat_bins)==0:
            bins[np.isneginf(bins)]=-1e32
            bins[np.isinf(bins)]=1e32
        else:
            bins[np.isneginf(bins)] = nat_bins.min()-(nat_bins.max()-nat_bins.min())*0.1
            bins[np.isinf(bins)] = nat_bins.max()+(nat_bins.max()-nat_bins.min())*0.1

        sort=np.argsort(bins)
        bins=bins[sort]
        counts = counts[sort]
        
        centroids = bins
        
        if not os.path.exists(path1) or recreate:
            plt.figure()
            counts_, bins_, _ = plt.hist(centroids, bins=min(5000,len(counts)),
                                         weights=counts, range=(min(bins), max(bins)))
            
            plt.title(basetitle+" (full)", fontdict = {'fontsize' : 8})
            plt.tight_layout()
            plt.savefig(path1,bbox_inches='tight')
            plt.close()
        
        for j in range(len(bin_sizes)):
            if os.path.exists(bin_paths[j]) and len(counts)<=bin_sizes[j]:
                os.remove(bin_paths[j])
                recreate=True
        
        for j in range(len(bin_sizes)):
            if os.path.exists(bin_paths[j]) and len(counts)<=bin_sizes[j]:
                os.remove(bin_paths[j])
            if (os.path.exists(bin_paths[j]) and not recreate) or len(counts)<=bin_sizes[j]:
                continue
                
            plt.figure()
            counts_, bins_, _ = plt.hist(centroids, bins=min(bin_sizes[j],len(counts)),
                                         weights=counts, range=(min(bins), max(bins)))
            
            plt.title(basetitle+f" (bins={bin_sizes[j]})", fontdict = {'fontsize' : 8})
            plt.tight_layout()
            plt.savefig(bin_paths[j],bbox_inches='tight')
            plt.close()

In [6]:
     
def check_ecdf_data_path(ecdf_data_path,dataloader,res=5000):
    if not os.path.exists(ecdf_data_path):
        return False,None
            
    with open(ecdf_data_path,"rb") as f:
        ecdf_data = pickle.load(f)

    if not isinstance(ecdf_data,dict):
        return False,ecdf_data

    if not "resolution" in ecdf_data:
        return False,ecdf_data
    
    if ecdf_data["resolution"]!=res:
        return False,ecdf_data
    
    if not "n_dists" in ecdf_data:
        return False,ecdf_data
    
    if not "data" in ecdf_data:
        return False,ecdf_data
    
    for d in ecdf_data["data"]:
        if not "smoothed" in d:
            return False,ecdf_data
        
        if not "unique" in d:
            return False,ecdf_data
        
    if "dataloader" not in ecdf_data:
        ecdf_data["dataloader"]=[]
    if dataloader is not None:
        for d in dataloader:
            if d not in ecdf_data["dataloader"]:
                ecdf_data["dataloader"].append(d)
        
    return True,ecdf_data

def histo_data_to_ecdf(histo_data,res=5000):
    ecdfs=[]
    print("histo data to ecdf")
    for i in tqdm(range(histo_data["n_dists"]),total=histo_data["n_dists"]):
        d=histo_data["histo_data"][i]
        unpacked_data = np.repeat(d["data"], d["counts"], axis=0)
        x1, y1 = generate_ecdf(unpacked_data,smooth=True,resolution_y=res)
        x2, y2 = generate_ecdf(unpacked_data,unique_only=True)
       # plt.figure(dpi=250)
       # plt.plot(x1,y1)
       # plt.plot(x2,y2,"--")
        #print(x1.shape,x2.shape)
        #plt.show()
        #plt.close()
        ecdfs.append({
            "smoothed":(x1, y1),
            "unique":(x2, y2),
        })
    return {
        "resolution":res,
        "data":ecdfs,
        "n_dists":histo_data["n_dists"]
    }

def check_ecdf_images(ecdf_data,ecdf_data_path,recreate=False,bin_sizes=[30,50],basetitle="histo"):
    ecdf_path=os.path.join(ecdf_data_path,"ecdf")
    os.makedirs(ecdf_path,exist_ok=True)
                
    pp=False
    for i in tqdm(range(ecdf_data["n_dists"]),total=ecdf_data["n_dists"]):
        path1=os.path.join(ecdf_path,f"{i}.png")
        if os.path.exists(path1) and not recreate:
            continue
            
        if not pp:
            print("gen ecdf images",ecdf_data_path)
            pp=True
            
        
        
        d=ecdf_data["data"][i]
        
        smoothed_x,smoothed_y = d["smoothed"]
        unique_x,unique_y = d["unique"]
        
        
        

        
        
        plt.figure()
        plt.plot(unique_x,unique_y,label="ecdf")
        plt.plot(smoothed_x,smoothed_y,"--",label="smoothed ecdf")
        
        plt.legend()
        plt.title(basetitle+" ECDF", fontdict = {'fontsize' : 8})
        plt.tight_layout()
        plt.savefig(path1,bbox_inches='tight')
        plt.close()
        

In [7]:
def merge_histo_data(histo_data_list):
    nd=histo_data_list[0]['n_dists']
    
   
    for hd in histo_data_list:
        if hd['n_dists']!=nd:
            raise ValueError("unequal dimensions")
    
    merged_data={
        'n_dists': nd,
        'dataloader': []
    }
    
    for hd in histo_data_list:
        merged_data['dataloader'].extend(hd['dataloader'])
    
    histo_data=[]
    for i in tqdm(range(nd),total=nd):
        unique,unique_inverse=np.unique(np.concatenate([hd['histo_data'][i]['data'] for hd in histo_data_list]),
                       return_inverse=True,
                      )
        counts=np.zeros_like(unique,dtype=int)
        concounts=np.concatenate([hd['histo_data'][i]['counts'] for hd in histo_data_list])
        for j,k in enumerate(unique_inverse):
            counts[k]+=concounts[j]
            
        histo_data.append(
            {'data': unique, 'counts': counts}
        )
        
    merged_data['histo_data']=histo_data
    
    return merged_data

    

In [8]:
def gen_histo_and_ecdf(featurizer):
    for idx in featurizer.index:
        merged_histo_data=[]
        change=False
        for dldir in found_dataloader_dirs:
            featpath=os.path.join(dldir,idx)
            basetitle=f"{os.path.basename(dldir)}\n{idx.replace('molNet.featurizer.','')}"
            if os.path.isdir(featpath):
                s_change=False
                outpath=os.path.join(OUT_DIR,os.path.relpath(featpath,BASEDIR))
                os.makedirs(outpath,exist_ok=True)

                histo_data_path=os.path.join(outpath,"histo_data.pckl")

                valid_histo,histo_data = check_histo_data_path(
                    histo_data_path,
                    dataloader=[os.path.basename(dldir)]
                )

                if not valid_histo:
                    s_change=True
                    histo_data = featpath_to_histo_data(featpath,histo_data_path)
                    histo_data["dataloader"]=[os.path.basename(dldir)]
                    with open(histo_data_path,"w+b") as f:
                        pickle.dump(histo_data,f)

                valid_histo,histo_data = check_histo_data_path(
                    histo_data_path,
                    dataloader=[os.path.basename(dldir)]
                )
                if not valid_histo:
                    continue

                check_histodata_images(histo_data,outpath,basetitle=basetitle,recreate=False)

                merged_histo_data.append(histo_data)

                ecdf_data_path=os.path.join(outpath,"ecdf_data.pckl")
                valid_ecdf,ecdf_data = check_ecdf_data_path(
                    ecdf_data_path,
                    dataloader=histo_data["dataloader"]
                )

                if not valid_ecdf or not valid_histo:
                    ecdf_data = histo_data_to_ecdf(histo_data)
                    s_change=True

                    with open(ecdf_data_path,"w+b") as f:
                        pickle.dump(ecdf_data,f)
                check_ecdf_images(ecdf_data,outpath,basetitle=basetitle,recreate=False)
                #    esdf_data=gen_esdf(histo_data)

                change=change or s_change


        if len(merged_histo_data)>0:
            outpath=os.path.join(OUT_DIR,"merged",idx)
            os.makedirs(outpath,exist_ok=True)


            merged_data_loader=[]
            for d in merged_histo_data:
                merged_data_loader.extend(d['dataloader'])

            histo_data_path=os.path.join(outpath,"histo_data.pckl")

            #merged_data_loader.pop(1)

            valid_histo,histo_data = check_histo_data_path(
                    histo_data_path,
                    dataloader=merged_data_loader
                )
            #if merged_histo_data[0]["n_dists"]>1:
            #    print(idx)
            #    break
            basetitle=f"{idx.replace('molNet.featurizer.','')}"
            if not valid_histo or change:
                #print(histo_data_path)
                #print([hd['n_dists'] for hd in merged_histo_data])
                print("merge histo",idx)
                histo_data=merge_histo_data(merged_histo_data)
                #print(idx)
                #print(valid_histo,histo_data)
                with open(histo_data_path,"w+b") as f:
                        pickle.dump(histo_data,f)

            check_histodata_images(histo_data,outpath,basetitle=basetitle,recreate=change and False)

            ecdf_data_path=os.path.join(outpath,"ecdf_data.pckl")
            valid_ecdf,ecdf_data = check_ecdf_data_path(
                ecdf_data_path,
                dataloader=histo_data["dataloader"]
            )

            if not valid_ecdf or not valid_histo:
                print(idx)
                ecdf_data = histo_data_to_ecdf(histo_data)
                with open(ecdf_data_path,"w+b") as f:
                    pickle.dump(ecdf_data,f)
            check_ecdf_images(ecdf_data,outpath,basetitle=basetitle,recreate=False)

                #break
                    #raise ValueError()
    #            try_generate_distribution(featpath)
    #            try_generate_ecdf(featpath)

            

In [18]:
gen_histo_and_ecdf(molNet.featurizer.get_molecule_featurizer_info())

100%|██████████| 2048/2048 [00:00<00:00, 99181.77it/s]


In [19]:
gen_histo_and_ecdf(molNet.featurizer.get_atom_featurizer_info())

100%|██████████| 1/1 [00:00<00:00, 7810.62it/s]


In [53]:
from molNet.featurizer.normalization import linear_norm, min_max_norm,sig_norm,dual_sig_norm,genlog_norm, tanh_norm, weibull_norm

NORMS=[linear_norm, min_max_norm,sig_norm,dual_sig_norm,tanh_norm,
       genlog_norm,weibull_norm
      ]
NORM_PARAMS_GETTER={
    #linear_norm:None,
#    min_max_norm: lambda x,y: (x.min(),x.max()),
   # sig_norm:None,
   # dual_sig_norm:None,
}

def sig_norm_def(x,y):
    m = x[np.abs(y-0.5).argmin()]
    diff = x[np.abs(y-0.99).argmin()]-m
    if diff<=0:
        diff=1e-12
    d=5/diff
    return m,d 

def dualsig_norm_def(x,y):
    m = x[np.abs(y-0.5).argmin()]
    diff = x[np.abs(y-0.99).argmin()]-m
    if diff<=0:
        diff=1e-32
    d=5/diff
    return m,d,d

def minmax_norm_def(x,y):
    xmin = x[np.abs(y-0.01).argmin()]
    xmax = x[np.abs(y-0.99).argmin()]
    return xmin,xmax

def linear_norm_def(x,y):
    xmin,xmax = minmax_norm_def(x,y)
    if xmin == xmax:
        xmax = xmin + 1e-12
    m=1/(xmax-xmin)
    c=-xmin/(xmax-xmin)
    return m,c

def tanh_norm_def(x,y):
    m=x[np.abs(y-0.01).argmin()]
    diff = x[np.abs(y-0.99).argmin()]-m
    if diff<=0:
        diff=1e-32
    d=5/diff
    return m,d

def genlog_norm_def(x,y):
    m,B=sig_norm_def(x,y)
    return B,m,1,1


def weibull_norm_def(x,y):
    m = x[np.abs(y-0.5).argmin()]
    x99= x[np.abs(y-0.99).argmin()]
    if x99 <= m:
        x99 = m + max(1e-12,(x.max()-x.min())/100)
        
    k=(x.max()-x.min())/(x99-m)
    if np.isinf(k):
        k=10
    if k<=0:
        k=1e-12
    
    m-=x.min()
    if m<=0:
        m=1e-12
    l=(np.log(2)**(1/k))/m
    if l<=0:
        l=1e-32
    return l,k

#"startingpoints"
NORM_DEFAULT_PARAMS_GETTER={
    linear_norm: linear_norm_def,
    min_max_norm: minmax_norm_def,
    sig_norm: sig_norm_def,
    dual_sig_norm:dualsig_norm_def,
    tanh_norm:tanh_norm_def,
    genlog_norm:genlog_norm_def,
    weibull_norm:weibull_norm_def,
}

NORM_BOUNDS={
    #linear_norm:None,
    min_max_norm: lambda x,y: (x.min(),x.max()),
    genlog_norm: lambda x,y: ([1e-32,-np.inf,0,1e-12],[np.inf,np.inf,np.inf,np.inf]),
    weibull_norm: lambda x,y: ([1e-32,1e-32],[np.inf,np.inf]),
   # sig_norm:None,
   # dual_sig_norm:None,
}


def check_entry_complete(row,n,pmin,pmax):
    s = n.__name__
    st=[s,f"{float(pmin*100)}-{float(pmax*100)}"]
    cols = [f for f in row.keys() if f[0]==st[0] and f[1]==st[1]]

    if len(cols)==0:
        return False

    if np.isnan(row[cols].values).any():
        return False
    
    if tuple(st+["R2"]) not in cols:
        return False
    
    return True
    
def norm_row_already_complete(row,ranges):
    for n in NORMS:
        for pmin,pmax in ranges:
            if not check_entry_complete(row,n,pmin,pmax):
                return False

    return True

def get_limited_data(wsmoothed_x,wsmoothed_y,pmin=0,pmax=1):
    wsmoothed_p=(wsmoothed_y>=pmin)&(wsmoothed_y<=pmax)
    wsmoothed_x_p=wsmoothed_x[wsmoothed_p]
    wsmoothed_y_p=wsmoothed_y[wsmoothed_p]
    return wsmoothed_x_p,wsmoothed_y_p

In [54]:
def get_popt(featurizer_norm,row_id,wsmoothed_x,wsmoothed_y,pmin=0,pmax=1,redo=None):
        wsmoothed_x_p,wsmoothed_y_p = get_limited_data(wsmoothed_x,wsmoothed_y,pmin=pmin,pmax=pmax)
        if redo is None:
            redo = []
        for n in NORMS:
            if check_entry_complete(featurizer_norm.loc[row_id],n,pmin,pmax) and n not in redo:
                continue
            s = n.__name__
            st=[n.__name__,f"{float(pmin*100)}-{float(pmax*100)}"]
            
            popt=None
            if n in NORM_PARAMS_GETTER:
                popt = NORM_PARAMS_GETTER[n](wsmoothed_x_p,wsmoothed_y_p)
                r = np.sqrt(np.abs(n(wsmoothed_x,*popt)-wsmoothed_y)).mean()
            else:
                for method in [None,'dogbox']:
                    try:
                        #print(n)
                        popt, pcov = curve_fit(n,
                                               wsmoothed_x_p,
                                               wsmoothed_y_p,
                                               p0=NORM_DEFAULT_PARAMS_GETTER[n](wsmoothed_x_p,wsmoothed_y_p) if n in NORM_DEFAULT_PARAMS_GETTER else None,
                                               bounds = NORM_BOUNDS[n](wsmoothed_x_p,wsmoothed_y_p) if n in NORM_BOUNDS else (-np.inf, np.inf),
                                               method=method,
                                              )
                        r = np.sqrt(np.abs(n(wsmoothed_x,*popt)-wsmoothed_y)).mean()
                        break
                    except (RuntimeError,ValueError,TypeError):
                        continue
                        
            if popt is not None:
                for k in range(len(popt)):
                    tk=tuple(st+[k])
                    if tk not in featurizer_norm.columns:
                        featurizer_norm[tk]=np.nan

                    featurizer_norm.loc[row_id,(tk,)]=float(popt[k])
                    
                tk=tuple(st+["R2"])
                if tk not in featurizer_norm.columns:
                        featurizer_norm[tk]=np.nan
                featurizer_norm.loc[row_id,(tk,)]=r

In [55]:
def create_feat_norm(featurizer,path,redo=[]):
    if redo is None:
        redo = []
            
    try:
        with open(path,"r+b") as f:
            featurizer_norm = pickle.load(f)
    except FileNotFoundError:
        featurizer_norm = pd.DataFrame()
    
    try:
        for idx in featurizer.index:
            outpath=os.path.join(OUT_DIR,"merged",idx)
            if not os.path.exists(outpath):
                continue

            ecdf_data_path=os.path.join(outpath,"ecdf_data.pckl")
            valid_ecdf,ecdf_data = check_ecdf_data_path(
                ecdf_data_path,dataloader=None
            )
            if not valid_ecdf:
                continue



            for i in tqdm(range(ecdf_data['n_dists']),total=ecdf_data['n_dists']):
                # create row if not already available
                if (idx,i) not in featurizer_norm.index:
                    featurizer_norm = featurizer_norm.append(pd.Series(name=(idx,i),dtype=float))



                ranges=[(0,1),(0.01,0.99)]

                if norm_row_already_complete(featurizer_norm.loc[(idx,i)],ranges=ranges) and len(redo)==0:
                    continue


                d=ecdf_data['data'][i]
                smoothed_x,smoothed_y=d['smoothed']
                if smoothed_x.min()==smoothed_x.max():
                    continue

                #print(idx,i)
                diff_y=np.diff(smoothed_y)

                weight=np.zeros_like(smoothed_y)
                weight[0]=diff_y[0]
                weight[-1]=diff_y[-1]
                means=(diff_y[:-1]+diff_y[1:])/2
                weight[1:-1]=means
                weight/=weight.min()
                if weight.max()>1000:
                    weight/=weight.max()/1000

                if len(smoothed_x)<1000:
                    _x=np.linspace(smoothed_x.min(),smoothed_x.max(),1000)
                    smoothed_y = np.interp(_x, smoothed_x, smoothed_y)
                    weight = np.interp(_x, smoothed_x, weight)
                    smoothed_x = _x

                weight=weight.astype(int)

                wsmoothed_x=np.repeat(smoothed_x,weight)
                wsmoothed_y=np.repeat(smoothed_y,weight)

                for pmin,pmax in ranges:
                    get_popt(featurizer_norm,(idx,i),wsmoothed_x,wsmoothed_y,pmin=pmin,pmax=pmax,redo=redo)
             #   wsmoothed_1_99=(wsmoothed_y>=0.01)&(wsmoothed_y<=0.99)
             #   wsmoothed_x_1_99=wsmoothed_x[wsmoothed_1_99]
             #   wsmoothed_y_1_99=wsmoothed_y[wsmoothed_1_99]
        
    finally:    
        with open(path,"w+b") as f:
            pickle.dump(featurizer_norm,f)
    
    return featurizer_norm 

In [56]:
featurizer_norm = create_feat_norm(molNet.featurizer.get_atom_featurizer_info(),"featurizer_norm.pd")

100%|██████████| 1/1 [00:00<00:00, 207.79it/s]


In [57]:
featurizer_norm = create_feat_norm(molNet.featurizer.get_molecule_featurizer_info(),"featurizer_norm.pd")

  9%|▉         | 184/2048 [00:11<02:00, 15.47it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
featurizer_norm

In [ ]:
featurizer = molNet.featurizer.get_molecule_featurizer_info()
ecdf_img_dir="ecdf_images"

for idx in featurizer.index:
    outpath=os.path.join(OUT_DIR,"merged",idx)
    if not os.path.exists(outpath):
        continue
        
    ecdf_data_path=os.path.join(outpath,"ecdf_data.pckl")
    valid_ecdf,ecdf_data = check_ecdf_data_path(
        ecdf_data_path,dataloader=None
    )
    if not valid_ecdf:
        coninue
        
    for i in tqdm(range(ecdf_data['n_dists']),total=ecdf_data['n_dists']):
        if (idx,i) not in featurizer_norm.index:
            continue
        
            
        d=ecdf_data['data'][i]
        smoothed_x,smoothed_y=d['smoothed']
        if smoothed_x.min()==smoothed_x.max():
            continue
        
        if len(smoothed_x)<1000:
            _x=np.linspace(smoothed_x.min(),smoothed_x.max(),1000)
            smoothed_y = np.interp(_x, smoothed_x, smoothed_y)
            smoothed_x = _x
        
#        cols=[f for f in featurizer_norm.columns if f[0]==s]
        ps=set()
        for s in featurizer_norm.columns:
            p1,p2 = s[1].split("-")
            ps.add((float(p1),float(p2)))
            
        row=featurizer_norm.loc[(idx,i)]
        
        for p in ps:
            p_string=f"{p[0]}-{p[1]}"
            imgdir=os.path.join(ecdf_img_dir,idx,p_string)
            impath=os.path.join(imgdir,f"{i}.png")
           # if os.path.exists(impath):
           #     continue
            
            lsmoothed_x,lsmoothed_y = get_limited_data(smoothed_x,smoothed_y,pmin=p[0]/100,pmax=p[1]/100)
       
            
        
        
            plt.figure()
            
            plt.plot(lsmoothed_x,lsmoothed_y,label="smoothed ecdf",linewidth=5, zorder=1)
            n_data=[]
            for n in NORMS:
                s=n.__name__
                
                cols=[f for f in featurizer_norm.columns if f[0]==s and f[1]==p_string]
                para_cols=[c for c in cols if c[2]!="R2"]
                rcol=[c for c in cols if c[2]=="R2"][0]
                
                para_cols = sorted(para_cols,key=lambda c:c[2])
                params=row[para_cols]
                if np.any(np.isnan(params)):
                    continue
                
                n_data.append((
                    n(lsmoothed_x,*params),
                    s,
                    row[rcol],
                ))
            
            n_data=sorted(n_data,key=lambda d:d[2])
            
            if len(n_data)>0:
                y,s,r = n_data[0]
                plt.plot(lsmoothed_x,y,"-",label=f"{s} (R2={r:.3f})",linewidth=1, zorder=len(n_data)+1)
                
            if len(n_data)>1:
                for l,(y,s,r) in enumerate(n_data[1:]):
                    plt.plot(lsmoothed_x,y,"--",label=f"{s} (R2={r:.3f})",zorder=len(n_data)-l)  
            
            
            
            
            plt.ylim(-0.1,1.1)
            
            plt.title(f"{idx} {i}\n({p_string}%)" )
            
            
            
            plt.legend()
            
            os.makedirs(imgdir,exist_ok=True)
            plt.tight_layout()
            plt.savefig(impath,bbox_inches='tight')
            plt.close()
        
        
    
        